# Bulk processor

In [21]:
from collections import namedtuple
from copy import deepcopy

from ssscoring import BREAKOFF_ALTITUDE
from ssscoring import PERFORMANCE_WINDOW_LENGTH
from ssscoring import aggregateResults
from ssscoring import convertFlySight2SSScoring
from ssscoring import dropNonSkydiveDataFrom
from ssscoring import getAllSpeedJumpFilesFrom
from ssscoring import getSpeedSkydiveFrom
from ssscoring import isValidJump
from ssscoring import jumpAnalysisTable
from ssscoring import processAllJumpFiles
from ssscoring import roundedAggregateResults
from ssscoring import totalResultsFrom
from ssscoring import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [22]:
DATA_LAKE_ROOT = './data'

In [23]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [24]:
jumpResults = processAllJumpFiles(jumpFiles)

---
## Results

In [25]:
aggregate = aggregateResults(jumpResults)
sumResults = totalResultsFrom(aggregate)
display(aggregate)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
09-54-07,445.05,172.152,304.128,396.252,437.220,440.496,25.0,448.164
10-36-56,411.354,173.088,308.124,388.872,409.248,0.000,25.0,412.056
11-55-36,432.864,168.732,298.692,382.572,435.276,370.332,25.0,441.720


### Rounded results for training log

In [26]:
roundedResults = roundedAggregateResults(jumpResults)

### All jumps

In [27]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [28]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [29]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)</h3>'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        hoverValue = bm.HoverTool(tooltips = [ ('KM/h', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">09-54-07 jump - 🟢 valid - score = 445.05 km/h</span></h1>')

HTML(value='<h3>Max speed = 448; exit at 4233 m (13890 ft), end scoring window at 1977 m (6489 ft)</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,172,112,57.0,205,13458
,10.0,304,80,75.3,314,12339
,15.0,396,66,80.6,402,10718
,20.0,437,39,84.9,439,8796
,25.0,440,13,88.4,441,6773


HTML(value='<hr><h1><span style="color: #0f0">10-36-56 jump - 🟢 valid - score = 411.35 km/h</span></h1>')

HTML(value='<h3>Max speed = 412; exit at 3882 m (12736 ft), end scoring window at 1707 m (5600 ft)</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,173,105,58.8,202,12038
,10.0,308,83,74.9,319,10729
,15.0,389,54,82.0,393,9010
,20.0,409,46,83.6,412,7076
,25.0,0,0,0.0,0,0


HTML(value='<hr><h1><span style="color: #0f0">11-55-36 jump - 🟢 valid - score = 432.86 km/h</span></h1>')

HTML(value='<h3>Max speed = 442; exit at 4067 m (13344 ft), end scoring window at 1811 m (5943 ft)</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,169,111,56.6,202,12920
,10.0,299,86,73.9,311,11818
,15.0,383,74,79.0,390,10238
,20.0,435,51,83.3,438,8355
,25.0,370,119,72.2,389,6525


---
---
## Competition summary

In [30]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
09-54-07,445,172,304,396,437,440,25.0,448
10-36-56,411,173,308,389,409,0,25.0,412
11-55-36,433,169,299,383,435,370,25.0,442


,totalSpeed,meanSpeed,maxScore
totalSpeed,1289.268,429.756,445.05
